In [0]:
%sql
use bricks_project.deltadb

##OPTIMIZE COMMAND

In [0]:
%sql
create or replace table logan_sales(sid int,region string,price double,product string) using delta;

In [0]:
%sql
INSERT INTO logan_sales VALUES
  (1,'North', 1000.50, 'mango'),
  (2,'South', 500.75, 'veg'),
  (3,'East', 700.20, 'ffd'),
  (4,'West', 1200.00, 'nathan');

INSERT INTO logan_sales VALUES
  (5,'North', 1000.50, 'fff'),
  (6,'South', 500.75, 'ddd'),
  (7,'East', 700.20, 'aaa'),
  (8,'West', 1200.00, 'vvv');

In [0]:
%sql
INSERT INTO logan_sales VALUES
  (9,'North', 1000.50, 'mango'),
  (10,'South', 500.75, 'veg'),
  (11,'East', 700.20, 'ffd'),
  (12,'West', 1200.00, 'nathan');

In [0]:
%sql
desc history logan_sales;

In [0]:
%sql
DESCRIBE detail logan_sales;

In [0]:
%sql
--Optimize the table
--This performs file compaction:
--Combines many small Parquet files into fewer large files (around 1 GB default).
--Improves read performance and reduces metadata overhead.
OPTIMIZE logan_sales;

In [0]:
%sql
use bricks_project.deltadb

In [0]:
%sql
-- The CLUSTER BY clause enables liquid clustering automatically.
CREATE TABLE IF NOT EXISTS sales_orders_liquid
(
  order_id INT,
  customer_id INT,
  region STRING,
  product STRING,
  quantity INT,
  price DOUBLE,
  order_date DATE
)
USING DELTA
CLUSTER BY (customer_id, region);--clustering column can be high or low cardinal, unlike partition which requires only low cardinal columns.
--column order used in cluster by is based on the primary filter, ie. whether you first filter based on customer_id or region, accordingly keep the coloumns order.

In [0]:
%sql
-- Each insert simulates separate data ingestion.

INSERT INTO sales_orders_liquid VALUES
 (1, 101, 'North', 'Laptop', 2, 65000, '2025-10-01'),
 (2, 102, 'South', 'Headphones', 5, 2500, '2025-10-01'),
 (3, 103, 'West', 'Desk Chair', 3, 4500, '2025-10-02');

INSERT INTO sales_orders_liquid VALUES
 (4, 101, 'North', 'Keyboard', 1, 1200, '2025-10-03'),
 (5, 104, 'East', 'Monitor', 2, 9500, '2025-10-03'),
 (6, 105, 'South', 'Mouse', 4, 700, '2025-10-03');


In [0]:
%sql
SELECT * FROM sales_orders_liquid;-- where customer_id=102;

In [0]:
%sql
UPDATE sales_orders_liquid
SET price = price * 1.05
WHERE region = 'North';

In [0]:
%sql
DELETE FROM sales_orders_liquid
WHERE region = 'East';

In [0]:
%sql
--check the clustering column
DESCRIBE detail sales_orders_liquid

In [0]:
%sql
desc history sales_orders_liquid;

In [0]:
%sql
CREATE OR REPLACE TABLE orders_dv AS
SELECT
  id AS order_id,
  CASE WHEN id % 2 = 0 THEN 'APAC' ELSE 'EMEA' END AS region
FROM range(0, 20);
select * from orders_dv;

In [0]:
%sql
ALTER TABLE orders_dv
SET TBLPROPERTIES ('delta.enableDeletionVectors' = true);

In [0]:
%sql
--look at the properties delta.enableDeletionVectors: "true"
DESCRIBE DETAIL orders_dv;

In [0]:
%sql
DELETE FROM orders_dv WHERE region = 'APAC';

In [0]:
%sql
--numDeletionVectorsAdded: "8" (no real deletes happened, just deletion vector added)
DESCRIBE HISTORY orders_dv

In [0]:
%sql
ALTER TABLE orders_dv
SET TBLPROPERTIES ('delta.enableDeletionVectors' = false);

In [0]:
%sql
DELETE FROM orders_dv WHERE order_id = 3;

In [0]:
%sql
DESCRIBE DETAIL orders_dv;

In [0]:
%sql
DESCRIBE HISTORY orders_dv;